<a href="https://colab.research.google.com/github/umutcan-dev/yapay-zeka-api/blob/main/python_api_ipynb_umutcan_revize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_ngrok #sunucuda çalıştırılmayacak - sadcece colab!
!pip install flask
!pip install flask_restful
#!pip install keras
#!pip install tensorflow
#!pip install pandas
#!pip install numpy

In [ ]:
from flask import Flask, request
from flask_restful import Api, Resource
from flask_ngrok import run_with_ngrok
import pandas as pd
from PIL import Image
import keras
from keras import backend as K
#from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils import img_to_array
from flask import jsonify
import io
import numpy as np
import base64
import tensorflow as tf
from tensorflow.keras import applications
from keras import applications
from tensorflow.keras.models import Sequential
from google.colab import drive #sadece colab
drive.mount('/content/drive') #sadece colab

Mounted at /content/drive


In [ ]:
!pip install --upgrade --no-cache-dir gdown

!gdown --id '1qZ5oy3NwjBqozIcc7JItdFjCqB25FEaM' #h5 download
!gdown --id '1YZNf6fFFoHuYEe-wf-93dOhe2kfuEPJE' #.csv file download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1qZ5oy3NwjBqozIcc7JItdFjCqB25FEaM
To: /content/model_weights.h5
100% 554M/554M [00:07<00:00, 74.3MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1YZNf6fFFoHuYEe-wf-93dOhe2kfuEPJE
To: /content/csvornek.csv
100% 29.0/29.0 [00:00<00:00, 45.5kB/s]


In [ ]:
app = Flask(__name__)
api = Api(app)

In [ ]:
global image

In [ ]:
def preprocess_image(image, target_size):
    if image.mode != "RGB":
        image = image.convert("RGB")
    
    image = image.resize(target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    print(np.shape(image))
    
    return image

In [ ]:
def get_model():
    global model
    global graph
    vgg16_model = applications.vgg16.VGG16()
    model = tf.keras.Sequential()


    for i in vgg16_model.layers:
        model.add(i)


    for layer in model.layers:
        layer.trainable = False


    model.add(Dense(4, activation='softmax'))
    model.make_predict_function('model_weights.h5')
    
    #model._make_predict_function()
    graph = tf.compat.v1.get_default_graph()
    print("Model loaded!")

In [ ]:
print(" Loading Keras model...")
get_model()

 Loading Keras model...
553467096/553467096 [==============================] - 7s 0us/step
Model loaded!


In [ ]:
#train_path = '/content/drive/MyDrive/yapay_zeka/valid'
#import pathlib
#train_data_dir = pathlib.Path(train_path)
#import tensorflow as tf
#train_ds = tf.keras.utils.image_dataset_from_directory(train_path)
#train_class_names = train_ds.class_names
#print(train_class_names)
#

Found 200 files belonging to 4 classes.
['apple', 'apricot', 'banana', 'carrot']


In [ ]:
import csv

# opening the CSV file
with open('csvornek.csv', mode ='r')as file:

# reading the CSV file
  csvFile = csv.reader(file)
  for lines in csvFile:
		  fileList = lines

In [ ]:
class NeYedim(Resource):
      def get(self):
        #name = request.args['name']
        #processed_image = preprocess_image(image, target_size=(224, 224))
        
        data = {
            'prediction': {
            'get' : 'get işlemi',
            'sonuc': fileList[id]
            }
        }

        
        print(data)
        return {'data' : data}, 200
      
      #post işlemi
      def post(self):
        name = request.form['name']
        encoded = request.form['image']
        decoded = base64.b64decode(encoded)
        decode = io.BytesIO(decoded)
        image = Image.open(decode)
        #processed_image = preprocess_image(image, target_size=(224, 224))


        image.save("picture.jpg")
        image = Image.open("picture.jpg")
        
        
        #import tensorflow as tf
        #image = Image.open("/content/drive/MyDrive/yapay_zeka/train/apple/3.jpg") 
        #image.save("/content/drive/MyDrive/yapay_zeka/train/apple/3.jpg")
        #image = Image.open("/content/drive/MyDrive/yapay_zeka/train/apple/3.jpg")   

        processed_image = preprocess_image(image, target_size=(224, 224))

        #graph = tf.compat.v1.get_default_graph()


        with graph.as_default():
            vgg16_model = applications.vgg16.VGG16()
            model = tf.keras.Sequential()
        
            for i in vgg16_model.layers:
                model.add(i)
        
            for layer in model.layers:
                layer.trainable = False
        
            model.add(Dense(4, activation='softmax'))
            #model.make_predict_function('/content/drive/MyDrive/model_weights.h5')  
            model.load_weights('/content/drive/MyDrive/model_weights.h5')
            prediction = model.predict(processed_image).tolist()
        
        #prediction = [[0,1,2,3]] #kodda bulunan hata düzeltildikten sonra bu alan yorum satırına alınacak ve üstte bulunan kod bloğu çalıştırılacak -- çalıştırıldı!!!
        print(prediction)


        #karşılaştırma sonucu elde edilen en yüksek doğruluk oranına sahip dosyanın indeksi bulunuyor.
        prediction_max = max(prediction[0])
        print(prediction_max)

        id = -1
        for i in range(4):
         if(prediction[0][i]==prediction_max):
           id=i
        print(id)


        data = {
            'prediction': {
            'name' :  name,
            'post' : 'post işlemi',
            'sonuc': fileList[id]
            }
        }

        
        print(data)
        return {'data' : data}, 200

In [ ]:
# Add URL endpoints
api.add_resource(NeYedim, '/neyedinanaliz')

In [ ]:
if __name__ == '__main__':
#     app.run(host="0.0.0.0", port=5000)
    run_with_ngrok(app) #colab dışında çalıştırılırsa yorum satırına alınmalı
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b46d-35-225-136-56.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [05/Jan/2023 21:49:58] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Jan/2023 21:49:58] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:__main__:Exception on /neyedinanaliz [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/lib/python3.8/dist-packages/flask_restful/__init__.py", line 467, in wrapper
    resp = resource(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/flask/views.py", line 89, in view
    return self.dispatch_request(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/flask_restful/__init__.py", line 582, in dispatch_request
    resp = meth(*args, **kwargs)
  File "<ipython-input-10-a7643b597754>", line 9, in get
    'sonuc'